I was unable to find any large dataset which contained user emails of the type I wanted.
Fitting any clustering algorithm on this dataset was producing satisfactory but not good results
hence, I discarded the idea.
however this notebook still contains much of the work which I tried.

In [79]:
import re

import joblib
import pandas as pd
import scipy.sparse
from sklearn.impute import SimpleImputer

mails_df = pd.read_csv("../data/training_data.csv", sep='~', index_col=0)
imputer = SimpleImputer(strategy='constant', fill_value='')
mails_df = pd.DataFrame(imputer.fit_transform(mails_df), columns=mails_df.columns)
# mails_df = mails_df.iloc[:10]

In [80]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
import re

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

In [81]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    processed_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
    return processed_text


def preprocess_sender(address):
    address_lst = address.lower().split('@')
    address_lst[1] = re.sub('[.]ac|[.]in|[.]com', '', address_lst[1])
    address_lst[1] = re.sub('[.]', ' ', address_lst[1])
    address_lst[0] = re.sub('[._]', '', address_lst[0])
    return ' '.join(address_lst)

preprocessor = ColumnTransformer(transformers=[
    ('subject', TfidfVectorizer(preprocessor=preprocess_text, min_df=0.01), 'subject'),
    ('body', TfidfVectorizer(preprocessor=preprocess_text, max_df=0.9, min_df=0.1), 'body'),
    ('sender', TfidfVectorizer(preprocessor=preprocess_sender), 'sender')
], remainder='drop')

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

In [82]:
preprocess_sender('hello_me@cs.iitr.ac.in')

'hellome cs iitr'

In [83]:
from sklearn.preprocessing import MultiLabelBinarizer
import json

with open('../data/label_dict.json', 'r') as file:
    all_labels = json.load(file)
label_list = [key for key in all_labels.keys() if re.match('Label_[0-9]', key)]
mlb = MultiLabelBinarizer(classes=label_list)
labels_array = [list(st.split(',')) for st in mails_df['labels']]
mlb.fit(label_list)
labels = mlb.transform(labels_array)

C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['CATEGORY_FORUMS', 'CATEGORY_PERSONAL', 'CATEGORY_PROMOTIONS', 'CATEGORY_UPDATES', 'IMPORTANT', 'INBOX', 'SENT', 'STARRED', 'UNREAD'] will be ignored
  warnings.warn(


In [84]:
feature_matrix = pipeline.fit_transform(mails_df)
feature_matrix

<1126x381 sparse matrix of type '<class 'numpy.float64'>'
	with 26770 stored elements in Compressed Sparse Row format>

In [ ]:
X = feature_matrix
y = labels
print(X.shape, y.shape)

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans

k = 25
model = KMeans(n_clusters=k, random_state=42)
labels = model.fit_predict(X)
labels

In [ ]:
type(labels)

In [ ]:
mails_df['label'] = pd.Series(labels, name='label')

In [ ]:
mails_df.loc[:,['subject', 'label']]

In [3]:
import dill, gzip
tfidf = dill.load(dill.load(gzip.open('../data/TfidfVectorizers.pkl', 'rb')))

BadGzipFile: Not a gzipped file (b'x\x01')